# Importações de biblilotecas

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore

# Exploração dos Dados

In [ ]:
train_data= pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
train_data.head()

In [ ]:
len(train_data)

In [ ]:
correlation_matrix = train_data.corr()
correlation_matrix['SalePrice']

In [ ]:
min(correlation_matrix['SalePrice'])

Como o menor valor para a correlação com o preço de venda da casa tem um módulo baixo, podemos nos preocupar apenas com os que possuem valores positivos. Para filtrar quais são os mais importantes, vamos considerar os que possuem correlação maior do que 0.5 com o preço de venda.

In [ ]:
labels = []
for i in range(len(correlation_matrix['SalePrice'])):
    if correlation_matrix['SalePrice'][i] > 0.5:
        labels.append(correlation_matrix['SalePrice'].index[i])

In [ ]:
labels

In [ ]:
new_train_data = train_data[labels]

new_train_data.head()

Todos são valores numéricos, o que já facilita bastante nosso problema! Mas ainda podemos tratar melhor esses dados. Primeiro, vamos procurar por outliers. Para isso, vamos usar um boxplot para cada parâmetro.

In [ ]:
for i in new_train_data.columns:
    sns.boxplot(new_train_data[i])
    plt.title(f'{i}')
    plt.show()

In [ ]:
#Procurando valores faltantes

new_train_data.isna().sum()

Com exceção de 2 parâmetros, todos os demais apresentam outliers. Importante observar que, provavelmente, alguns deles se referem ao mesmo dado. Vamos remover todos para termos dados mais suaves para o modelo treinar.

In [ ]:
train_zscores = zscore(new_train_data)
abs_zscores = np.abs(train_zscores)
zscore_filter = (abs_zscores<3).all(axis=1)

new_train_data=new_train_data[zscore_filter]
len(new_train_data)

Foram eliminados apenas 54 amostras, parece uma perda razoável. Agora, podemos normalizar os dados e passar para o treinamento dos modelos. Não podemos esquecer de tirar o valor valor de venda da casa antes de normalizar os dados!

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
y_train = new_train_data.pop('SalePrice')

In [ ]:
x = new_train_data.values
min_max = MinMaxScaler()
x_scaled = min_max.fit_transform(x)
x_train = pd.DataFrame(x_scaled)

x_train.head()

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
forest = RandomForestRegressor(random_state=0)
forest.fit(x_train, y_train)

In [ ]:
forest.score(x_train, y_train)

In [ ]:
forest.feature_importances_

In [ ]:
print(f'Feature com maior importância de acordo com o modelo: {labels[0]}')

A floresta conseguiu descrever bem os dados de treino e a featura a qual ela deu maior importância é a mesma que possui maior correlação com o preço, de acordo com nossa matriz de correlação.

Agora precisamos separar as features importantes nos dados de teste e colocar a floresta para fazer uma previsão.

In [ ]:
labels.pop(-1)

In [ ]:
x_test = test_data[labels]
x_test.isna().sum()

Valores faltantes para garagem e e porão podem indicar que essas casas simplesmente não possuem esses cômodos. Por isso, vamos preencher seus valores com 0.

In [ ]:
x_test = x_test.fillna(0)
x_test.isna().sum()

In [ ]:
x = x_test.values
min_max = MinMaxScaler()
x_scaled = min_max.fit_transform(x)
x_test = pd.DataFrame(x_scaled)

x_test.head()

In [ ]:
predictions = forest.predict(x_test)

Agora vamos submeter as predições e ver nosso resultado

In [ ]:
output = pd.DataFrame({'Id': x_test.index+1461,
                       'SalePrice': predictions})
output.to_csv('submission.csv', index=False)

Score obtido: 0.34482

# LightGBM

In [ ]:
import lightgbm as lgb

In [ ]:
light_regressor = lgb.LGBMRegressor()

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_pretest, y_train, y_pretest = train_test_split(x_train, y_train, test_size=0.2, random_state=0)

In [ ]:
light_regressor.fit(x_train, y_train, eval_set=[(x_pretest, y_pretest)])

In [ ]:
light_pred = light_regressor.predict(x_test)

In [ ]:
output_light = pd.DataFrame({'Id': x_test.index+1461,
                       'SalePrice': light_pred})
output_light.to_csv('light_submission.csv', index=False)

In [ ]:
output_light

Score: 0.33760

# Ensemble

In [ ]:
from sklearn.metrics import mean_squared_error

light_pred_train = light_regressor.predict(x_train)
forest_pred_train = forest.predict(x_train)

light_error = np.sqrt(mean_squared_error(light_pred_train, y_train))
forest_error = np.sqrt(mean_squared_error(forest_pred_train, y_train))

print(light_error)
print(forest_error)

In [ ]:
pred_df = pd.DataFrame({'forest': forest_pred_train, 'light':light_pred_train})

In [ ]:
#Os pesos foram dados de forma a dar mais valor ao modelo com menor loss.

forest_weight = 1 - forest_error/(forest_error+light_error)
light_weight = 1 - forest_weight

pred_df['ensemble'] = forest_weight*pred_df['forest'] + light_weight*pred_df['light']

print(np.sqrt(mean_squared_error(pred_df['ensemble'], y_train)))